In [ ]:
import io
import numpy as np
import pandas as pd
from copy import deepcopy
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
uploaded = files.upload()

In [ ]:
data_df = pd.read_csv(io.BytesIO(uploaded["njr_data.csv"]))
data_df

In [ ]:
data_df[data_df["team"].str.contains("Barcelona")]

In [ ]:
mins_df = data_df.groupby(["team"])["time"].sum().reset_index()
px.bar(x = "team",y = "time" , text = "time", data_frame = mins_df)

In [ ]:
data_df.info()

In [ ]:
data_df["goalsPer90"] = data_df["goals"].divide(data_df["time"]).multiply(90)
data_df["assistsPer90"] = data_df["assists"].divide(data_df["time"]).multiply(90)
per90Cols = ["goals", "assists", "xGChain",  "xGBuildup",
             "npxG", "npg", 'xG',"xA"]
for col in per90Cols:
    data_df[col + "Per90"] = data_df[col].divide(data_df["time"]).multiply(90)
cols_for_radar = [i + "Per90" for i in per90Cols]
print(cols_for_radar)

In [ ]:
data_df.loc[(data_df["team"] == "Paris Saint Germain"), cols_for_radar].sum()

In [ ]:
data_df.loc[(data_df["team"] == "Barcelona"), cols_for_radar].sum()

In [ ]:
fig = px.line_polar(data_df, r=data_df.loc[(data_df["team"] == "Barcelona"), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [ ]:
fig = px.line_polar(data_df, r=data_df.loc[(data_df["team"] == "Paris Saint Germain"), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_df[cols_for_radar] = scaler.fit_transform(data_df[cols_for_radar])

In [ ]:
fig = px.line_polar(data_df, r=data_df.loc[(data_df["player"] == "Neymar"), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [ ]:
for i, row in data_df.iterrows():
    print(row["team"])
    fig = px.line_polar(data_df, r=data_df.loc[(data_df["team"] == row["team"]), cols_for_radar].sum(),
                        theta=cols_for_radar, line_close=True,
                        title=row["team"])
    fig.update_traces(fill='toself')
    fig.show()

In [ ]:
id_names = data_df.drop_duplicates(subset=["id"])[["id", "player_name"]]
id_names.shape
id_names.head()
df_max = data_df[cols_for_radar].max().max()

In [ ]:
import plotly.graph_objects as go
for i, row in id_names.iterrows():
    if row["id"] == 647:
        continue
    print(row["player_name"])
    # Initiate the plotly go figure
    fig = go.Figure()
    # Add Radar plots for different players:
    fig.add_trace(go.Scatterpolar(
        r=data_df.loc[(data_df["id"] == 647), cols_for_radar].sum(),
        theta=cols_for_radar,
        fill='toself',
        name='Harry Kane'))
    fig.add_trace(go.Scatterpolar(
        r=data_df.loc[(data_df["id"] == row["id"]), cols_for_radar].sum(),
        theta=cols_for_radar,
        fill='toself',
        name=row["player_name"]))
    # Additional properties for the plot:
    fig.update_layout(
        title="Harry Kane vs " + row["player_name"],
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[0, df_max]
        )),
    showlegend=True
    )
    fig.show()